In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.applications import MobileNetV2
import matplotlib.pyplot as plt
import os

In [2]:
BATCH_SIZE = 32
IMAGE_SIZE = 256

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "D:/IIT Docs/2nd Year/Semester 1/CM 2603 - Data Science Project/Data/Dataset/Potato/color",
    seed=123,
    shuffle=True,
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 2152 files belonging to 3 classes.


In [4]:
# Preprocess dataset
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
dataset = dataset.map(lambda x, y: (preprocess_input(x), y))

In [5]:
# Split dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

In [6]:
train_dataset = dataset.take(train_size)
test_dataset = dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [12]:
# Create base model (MobileNetV2)
base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                         include_top=False,
                         weights='imagenet')

C:\Users\ASUS\AppData\Local\Temp\ipykernel_14640\3052378845.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),


In [13]:
# Freeze base model
base_model.trainable = False

In [14]:
# Add classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(3, activation='softmax')

In [15]:
# Build the model
inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = global_average_layer(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [16]:
# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [17]:
# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=8)

Epoch 1/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 42s 622ms/step - accuracy: 0.7158 - loss: 0.6534 - val_accuracy: 0.9526 - val_loss: 0.1709
Epoch 2/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 606ms/step - accuracy: 0.9533 - loss: 0.1566 - val_accuracy: 0.9828 - val_loss: 0.1271
Epoch 3/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 601ms/step - accuracy: 0.9690 - loss: 0.1095 - val_accuracy: 0.9698 - val_loss: 0.0916
Epoch 4/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 602ms/step - accuracy: 0.9781 - loss: 0.0828 - val_accuracy: 0.9784 - val_loss: 0.0782
Epoch 5/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 604ms/step - accuracy: 0.9802 - loss: 0.0680 - val_accuracy: 0.9741 - val_loss: 0.1053
Epoch 6/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 604ms/step - accuracy: 0.9831 - loss: 0.0623 - val_accuracy: 0.9871 - val_loss: 0.0621
Epoch 7/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 601ms/step - accuracy: 0.9906 - loss: 0.0514 - val_accuracy: 0.9828 - val_loss: 0.0740
Epoch 8/8
54/54 ━━━━━━━━━━━━━━━━━━━━ 33s 608ms/step - accuracy: 0.9974 - loss: 0.0414 - val_accuracy: 0.

In [18]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test accuracy: {accuracy}')
print("Accuracy: ", accuracy * 100, "%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 518ms/step - accuracy: 0.9725 - loss: 0.0709
Test accuracy: 0.984375
Accuracy:  98.4375 %


In [23]:
# Save the model
model_version = max([int(i) for i in os.listdir("../model_assets") + [0]]) + 1
model.save(f"../model_assets/{model_version}")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `tf.saved_model.save()` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=../model_assets/1.

In [24]:
model.save("../models/potato_mobilenetv2.h5")

ValueError: Unable to synchronously create dataset (name already exists)

In [18]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("potato_mobilenetv2.tflite", 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpn1ikpmsv\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmpn1ikpmsv\assets


In [6]:
import tensorflow

# modelConvert=tf.keras.models.load_model('riceModel.h5')
mc = tf.lite.Interpreter(model_path='riceModel.h5')
tflite_converter = tensorflow.lite.TFLiteConverter.from_keras_model(mc)

#convert
tflite_model = tflite_converter.convert()
open("rice_model.tflite", "wb").write(tflite_model)

ValueError: Model provided has model identifier '

', should be 'TFL3'


In [2]:
import tensorflow_hub as hub
custom_object = {'KerasLayer': hub.KerasLayer}

modelPath = 'riceModel.h5'
model = tf.keras.models.load_model(modelPath, custom_objects=custom_object)

ModuleNotFoundError: No module named 'keras.src.engine.base_layer_v1'